In [8]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
tf.config.list_physical_devices('GPU')
# tf.sysconfig.get_build_info() 

[]

In [1]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

2024-09-03 15:33:18.566351: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-03 15:33:18.567795: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 15:33:18.587839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 15:33:18.587861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 15:33:18.588670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
train_dataset_path = "/media/sombrali/HDD1/3d_object_detection/mediapipe/dataset/headband-autogen-dataset-coco/train"
validation_dataset_path = "/media/sombrali/HDD1/3d_object_detection/mediapipe/dataset/headband-autogen-dataset-coco/val"

train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 200
INFO:tensorflow:On image 300
INFO:tensorflow:On image 400
train_data size:  1580
validation_data size:  401


In [5]:
spec = object_detector.SupportedModels.MOBILENET_V2_I320
hparams = object_detector.HParams(batch_size=16, learning_rate = 0.01, cosine_decay_epochs = 20, epochs = 100, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

In [6]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

/home/sombrali/anaconda3/envs/mediapipe/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 80, 80, 24   2257984   
                             ),                                  
                              '3': (None, 40, 40, 32             
                             ),                                  
                              '4': (None, 20, 20, 96             
                             ),                                  
                              '5': (None, 10, 10, 32             
                             0),                                 
                              '6': (None, 10, 10, 12             
                             80)}                                
                                                                 
 fpn (FPN)                   {'5': (None, 10, 10, 12   130880    
                             8),                  

INFO:tensorflow:Training the models...


Epoch 1/100


/home/sombrali/anaconda3/envs/mediapipe/lib/python3.9/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


98/98 [==============================] - 75s 674ms/step - total_loss: 5.3445 - cls_loss: 5.0315 - box_loss: 0.0051 - model_loss: 5.2877 - val_total_loss: 1.5939 - val_cls_loss: 1.3367 - val_box_loss: 0.0040 - val_model_loss: 1.5371
Epoch 2/100
98/98 [==============================] - 64s 658ms/step - total_loss: 1.5526 - cls_loss: 1.2624 - box_loss: 0.0047 - model_loss: 1.4958 - val_total_loss: 1.5075 - val_cls_loss: 1.2402 - val_box_loss: 0.0042 - val_model_loss: 1.4508
Epoch 3/100
98/98 [==============================] - 65s 661ms/step - total_loss: 1.4526 - cls_loss: 1.1945 - box_loss: 0.0040 - model_loss: 1.3958 - val_total_loss: 1.4314 - val_cls_loss: 1.1857 - val_box_loss: 0.0038 - val_model_loss: 1.3747
Epoch 4/100
98/98 [==============================] - 65s 661ms/step - total_loss: 1.3860 - cls_loss: 1.1530 - box_loss: 0.0035 - model_loss: 1.3292 - val_total_loss: 1.3650 - val_cls_loss: 1.1419 - val_box_loss: 0.0033 - val_model_loss: 1.3082
Epoch 5/100
98/98 [=================

In [7]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

101/101 [==============================] - 3s 26ms/step - total_loss: 0.2908 - cls_loss: 0.1835 - box_loss: 0.0010 - model_loss: 0.2342
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.951
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.882
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.150
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.538
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.829
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.797
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.846
 Average Recall     (AR) @[

In [8]:
model.export_model()

Exporting a floating point model


/home/sombrali/anaconda3/envs/mediapipe/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmp0hv_fv47/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp0hv_fv47/saved_model/assets
2024-09-03 17:30:54.325476: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-03 17:30:54.325496: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-03 17:30:54.325670: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0hv_fv47/saved_model
2024-09-03 17:30:54.362019: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-03 17:30:54.362038: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp0hv_fv47/saved_model
2024-09-03 17:30:54.427399: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-03 17:30:54.448855: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-03 17:30:54.826996: I tensorflow/cc/saved_model/

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model.tflite


In [16]:
# qat_hparams = object_detector.QATHParams(learning_rate = 0.2, batch_size = 16, epochs = 20, decay_steps=6, decay_rate=0.96)
qat_hparams = object_detector.QATHParams(batch_size=32, learning_rate = 0.1, decay_steps = 20, decay_rate = 0.96, epochs = 100)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i320
Model: "retina_net_model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_8 (MobileNet)    {'2': (None, 80, 80, 24   2257984   
                             ),                                  
                              '3': (None, 40, 40, 32             
                             ),                                  
                              '4': (None, 20, 20, 96             
                             ),                                  
                              '5': (None, 10, 10, 32             
                             0),                                 
                              '6': (None, 10, 10, 12             
                             80)}                                
                                                                 
 fpn_8 (FPN)                 {'5': (Non

/home/sombrali/anaconda3/envs/mediapipe/lib/python3.9/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


49/49 [==============================] - 112s 2s/step - total_loss: 1.4659 - cls_loss: 0.8502 - box_loss: 0.0123 - model_loss: 1.4659 - val_total_loss: 1.8655 - val_cls_loss: 1.2668 - val_box_loss: 0.0120 - val_model_loss: 1.8655
Epoch 2/100
49/49 [==============================] - 91s 2s/step - total_loss: 0.8486 - cls_loss: 0.5661 - box_loss: 0.0057 - model_loss: 0.8486 - val_total_loss: 1.7048 - val_cls_loss: 1.1551 - val_box_loss: 0.0110 - val_model_loss: 1.7048
Epoch 3/100
49/49 [==============================] - 91s 2s/step - total_loss: 0.7044 - cls_loss: 0.4891 - box_loss: 0.0043 - model_loss: 0.7044 - val_total_loss: 1.3483 - val_cls_loss: 0.9913 - val_box_loss: 0.0071 - val_model_loss: 1.3483
Epoch 4/100
49/49 [==============================] - 91s 2s/step - total_loss: 0.6276 - cls_loss: 0.4433 - box_loss: 0.0037 - model_loss: 0.6276 - val_total_loss: 0.9973 - val_cls_loss: 0.7497 - val_box_loss: 0.0050 - val_model_loss: 0.9973
Epoch 5/100
49/49 [============================

In [17]:
model.export_model('model_int8_qat.tflite')

Exporting a qat int8 model


/home/sombrali/anaconda3/envs/mediapipe/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmpfs11jpu_/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpfs11jpu_/saved_model/assets
2024-09-04 10:52:51.593998: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-04 10:52:51.594018: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-04 10:52:51.594127: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpfs11jpu_/saved_model
2024-09-04 10:52:51.650031: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-04 10:52:51.650054: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpfs11jpu_/saved_model
2024-09-04 10:52:51.785806: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-04 10:52:52.622245: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpfs11jpu_/saved_model
2024-09-04 10:52:52.898686: I ten

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_int8_qat.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_int8_qat.tflite


In [12]:
from mediapipe_model_maker import quantization
quantization_config = quantization.QuantizationConfig.for_float16()
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)

Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i320
Model: "retina_net_model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_5 (MobileNet)    {'2': (None, 80, 80, 24   2257984   
                             ),                                  
                              '3': (None, 40, 40, 32             
                             ),                                  
                              '4': (None, 20, 20, 96             
                             ),                                  
                              '5': (None, 10, 10, 32             
                             0),                                 
                              '6': (None, 10, 10, 12             
                             80)}                                
                                                                 
 fpn_5 (FPN)                 {'5': (None

/home/sombrali/anaconda3/envs/mediapipe/lib/python3.9/site-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tmpmxrcke57/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpmxrcke57/saved_model/assets
2024-09-03 18:11:59.533295: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-03 18:11:59.533316: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-03 18:11:59.533419: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpmxrcke57/saved_model
2024-09-03 18:11:59.572008: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-03 18:11:59.572029: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpmxrcke57/saved_model
2024-09-03 18:11:59.663285: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-03 18:12:00.046804: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpmxrcke57/saved_model
2024-09-03 18:12:00.218674: I ten

INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_fp16.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully to: exported_model/model_fp16.tflite
